In [1]:
import requests

In [13]:
import pandas as pd
from pandas import json_normalize

info_params = {
    'id': '1077894',
}

info_url = 'https://www.fotmob.com/api/playerData'

# Load data using json normalize
# The point of the method and extra code is to ensure that the data is loaded consistently every time
def load_player_info(url , params):
    with requests.Session() as session:
        try:
            # Get the player information
            response = session.get(info_url , params = params)
            response.raise_for_status()
            data = response.json()

            # Extract player information
            player_info = data.get('playerInformation' , {})
            info_df = json_normalize(player_info)

            # Extract player name
            player_name = data.get('name')
            return info_df , player_name  # Return both DataFrame and player name as a tuple
        except requests.exceptions.RequestException as e:
            print(f"Error loading data: {e}")
    return pd.DataFrame() , None  # Return empty DataFrame and None for player name if there's an error

# Call load_data method and drop all NaN values for columns that meet the threshold
info_df , player_name = load_player_info(info_url , info_params)
info_df = info_df.dropna(axis = 1 , thresh = len(info_df) * 0.9)

# Rename columns and drop translationKey column
info_df = info_df.rename(columns={'title' : 'Title' , 'value.fallback' : 'Info'})
info_df = info_df.drop(columns = ['translationKey'])

# Add player's name as a new row
info_df.loc[len(info_df)] = ['Name' , player_name]

file_name = f"{player_name.replace(' ', '_')}_Player_Info.csv"
info_df.to_csv(file_name , index = False , encoding = 'utf-8-sig')

info_df

,Title,Info
0,Height,186 cm
1,Shirt,5
2,Age,20
3,Preferred foot,Right
4,Country,England
5,Market value,€115M
6,Name,Jude Bellingham


In [14]:
# change url and params due to the data being in a different api
stats_params = {
    'playerId': '1077894',
    'seasonId': '2023/2024-87',
}

stats_url = 'https://www.fotmob.com/api/playerStats'

# repeat the method used to get the player data
def load_stats(url , params):
    with requests.Session() as session:
        try:
            response = session.get(url , params = params)
            response.raise_for_status()
            data = response.json()

            # since the data is nested multiple times, we will run a for loop to iterate over all of the nested keys
            player_stats = data.get('statsSection' , {}).get('items' , [])
            all_stats = []
            for item in player_stats:
                inner_key = item.get('items' , [])
                for item in inner_key:
                    all_stats.append(item)

            stats_df = json_normalize(all_stats)

            return stats_df
        except requests.exceptions.RequestException as e:
            print(f"Error loading data: {e}")
    return pd.DataFrame()

stats_df = load_stats(stats_url , stats_params)
stats_df = stats_df.rename(columns = {'title' : 'Title' , 'localizedTitleId' : 'Text' , 'statValue' : 'Stat' , 'per90' : 'statPer90' , 'percentileRank' : 'Percentile' , 'percentileRankPer90' : 'percentilePer90'})
stats_df = stats_df.drop(columns = ['statFormat'])

file_name = f"{player_name.replace(' ', '_')}_Player_Stats.csv"
stats_df.to_csv(file_name , index = False , encoding = 'utf-8-sig')

stats_df

,Title,Text,Stat,statPer90,Percentile,percentilePer90
0,Goals,goals,19,0.736117,100.000000,95.890411
1,xG,expected_goals,11.12,0.430702,100.000000,89.041096
2,xGOT,expected_goals_on_target,13.65,0.528660,100.000000,94.520548
3,Penalty goals,goals_subtitle,1,0.038743,87.671233,90.410959
4,xG excl. penalty,non_penalty_xg,10.33,0.400157,100.000000,87.671233
5,Shots,shots,63,2.440809,90.410959,69.863014
6,Shots on target,ShotsOnTarget,35,1.356005,97.260274,89.041096
7,Assists,assists,6,0.232458,91.780822,80.821918
8,xA,expected_assists,4.31,0.166835,84.931507,67.123288
9,Accurate passes,successful_passes,1347,52.186827,100.000000,97.260274


In [11]:
stats_params = {
    'playerId': '1077894',
    'seasonId': '2023/2024-87',
}

stats_url = 'https://www.fotmob.com/api/playerStats'

# repeat the method used to get the player data
def load_shotmap(url, params):
    with requests.Session() as session:
        try:
            response = session.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            shotmap = data.get('shotmap' , {})
            shots_df = json_normalize(shotmap)

            return shots_df
        except requests.exceptions.RequestException as e:
            print(f"Error loading data: {e}")
    return pd.DataFrame()

shots_df = load_shotmap(stats_url, stats_params)
shots_df = shots_df.drop(columns = ['goalCrossedY' , 'goalCrossedZ' , 'homeTeamId' , 'homeScore' , 'awayTeamId' , 'id' , 'playerId' , 'playerName' , 'teamColor' , 'teamColorDark' , 'teamId'])
shots_df = shots_df.drop(columns = ['onGoalShot.zoomRatio' , 'minAdded' , 'awayScore'])
shots_df = shots_df.drop(columns = ['box'])
shots_df = shots_df.drop(columns = ['isSavedOffLine'])
shots_df = shots_df.drop(columns = ['matchId'])

shots_df = shots_df.rename(columns = {
    'eventType' : 'Type' ,
    'min' : 'Minute' ,
    'expectedGoals' : 'xG' ,
    'expectedGoalsOnTarget' : 'onTargetxG' ,
    'isFromInsideBox' : 'insideBox' ,
    'onGoalShot.x' : 'onGoalShotX' ,
    'onGoalShot.y' : 'onGoalShotY' ,
    'period' : 'Half'
})

file_name = f"{player_name.replace(' ', '_')}_Player_Shots.csv"
shots_df.to_csv(file_name , index = False , encoding = 'utf-8-sig')

shots_df

,Type,x,y,Minute,isBlocked,isOnTarget,blockedX,blockedY,xG,onTargetxG,shotType,situation,Half,isOwnGoal,insideBox,homeTeamName,awayTeamName,matchDate,onGoalShotX,onGoalShotY
0,AttemptSaved,80.232524,17.833631,8,False,True,103.482759,32.246250,0.020178,0.1160,RightFoot,RegularPlay,FirstHalf,False,False,Al Nassr FC,Al-Taawoun,2023-08-18T18:00:00Z,0.495701,0.193651
1,AttemptSaved,77.079126,53.879745,40,True,True,89.500000,46.172739,0.026094,NaN,RightFoot,FreeKick,FirstHalf,False,False,Al Nassr FC,Al-Taawoun,2023-08-18T18:00:00Z,0.616733,0.322751
2,Miss,100.448276,37.202500,46,False,False,NaN,NaN,0.385608,NaN,Header,FromCorner,SecondHalf,False,True,Al Nassr FC,Al-Taawoun,2023-08-18T18:00:00Z,0.096561,0.677249
3,AttemptSaved,86.997059,32.627500,49,False,True,102.724138,34.000000,0.082023,0.0984,RightFoot,RegularPlay,SecondHalf,False,False,Al Nassr FC,Al-Taawoun,2023-08-18T18:00:00Z,1.403439,0.064550
4,Miss,94.385965,33.923750,13,False,False,NaN,NaN,0.081436,NaN,Header,RegularPlay,FirstHalf,False,True,Al Fateh FC,Al Nassr FC,2023-08-25T18:00:00Z,1.062709,0.677249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,Goal,97.473684,41.945937,45,False,True,NaN,NaN,0.227535,0.3283,LeftFoot,RegularPlay,FirstHalf,False,True,Al Nassr FC,Al Ittihad,2024-05-27T18:00:00Z,0.717593,0.032275
177,AttemptSaved,94.385965,31.102500,45,False,True,98.245614,32.322500,0.479242,0.7627,RightFoot,RegularPlay,FirstHalf,False,True,Al Nassr FC,Al Ittihad,2024-05-27T18:00:00Z,0.737765,0.022083
178,Miss,98.342105,49.185478,57,False,False,NaN,NaN,0.053035,NaN,LeftFoot,RegularPlay,SecondHalf,False,True,Al Nassr FC,Al Ittihad,2024-05-27T18:00:00Z,2.000000,0.006060
179,AttemptSaved,85.601471,42.813125,68,True,True,92.400000,39.460952,0.077345,NaN,RightFoot,FreeKick,SecondHalf,False,False,Al Nassr FC,Al Ittihad,2024-05-27T18:00:00Z,1.342923,0.322751
